In [69]:

from urllib2 import Request, urlopen, URLError
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import folium
import matplotlib.cm as cm
import numpy as np
import matplotlib.colors as colors


## Foursquare API call function

### Testing the API

In [178]:
fs_api_call = "https://api.foursquare.com/v2/venues/search?client_id=IRTQBGCEC2CBZPY3JUR35GNMFP2TBEE5A3MJSRFRE0GZCHXX&\
client_secret=ICGW5IXET5QQLCBUJIQX4T4Q30SJB2YHTOVF1OF13JVNCPD4&v=20170511&ll=40.7347529,-73.8228129&query=park&limit=5&radius=3000"

In [179]:
results = requests.get(fs_api_call).json()

In [180]:
# assign relevant part of JSON to venues
venues = results["response"]["venues"]

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,beenHere.lastCheckinExpiredAt,categories,contact.facebook,contact.facebookName,contact.facebookUsername,contact.twitter,hasPerk,hereNow.count,hereNow.groups,hereNow.summary,...,referralId,specials.count,specials.items,stats.checkinsCount,stats.tipCount,stats.usersCount,url,venueChains,venueRatingBlacklisted,verified
0,0,"[{u'pluralName': u'Basketball Courts', u'prima...",104475634308,Bartow-Pell Mansion Museum,BartowPell,nycparks,False,0,[],Nobody here,...,v-1494774489,0,[],974,4,300,http://nyc.gov/parks,[{u'id': u'556df7e1a7c82e6b724d822e'}],NaN,True
1,0,"[{u'pluralName': u'Playgrounds', u'primary': T...",NaN,NaN,NaN,NaN,False,0,[],Nobody here,...,v-1494774489,0,[],678,2,129,NaN,[],NaN,False
2,0,"[{u'pluralName': u'Parks', u'primary': True, u...",NaN,NaN,NaN,NaN,False,0,[],Nobody here,...,v-1494774489,0,[],1029,1,216,NaN,[],NaN,False
3,0,"[{u'pluralName': u'Parks', u'primary': True, u...",NaN,NaN,NaN,nycparks,False,1,"[{u'count': 1, u'items': [], u'type': u'others...",One other person is here,...,v-1494774489,0,[],7036,25,1671,http://www.nycgovparks.org/parks/juniper-valle...,[{u'id': u'556df7e1a7c82e6b724d822e'}],NaN,True
4,0,[{u'pluralName': u'Residential Buildings (Apar...,NaN,NaN,NaN,NaN,False,0,[],Nobody here,...,v-1494774489,0,[],1741,0,133,NaN,[],True,False


In [177]:
dataframe.loc[:,'name']

0          Big Bush Park
1           Big Six Park
2        Windmuller Park
3    Juniper Valley Park
4            Celtic Park
Name: name, dtype: object

In [158]:
dataframe.loc[:,['location.lat','location.lng']]

,location.lat,location.lng
0,40.740713,-73.842931
1,40.747715,-73.808877
2,40.747907,-73.821774
3,40.730030,-73.861883
4,40.726169,-73.852866


## Breezometer API call

## testing the API

In [124]:
def get_row(api_call):
    request = Request(api_call)
    response = urlopen(request)
    info = response.read()
    row = pd.read_json(info, typ='series', orient='values')
    return row

In [127]:
breeze_input = 'https://api.breezometer.com/baqi/?lat={}&lon={}&key=932dc0cc968842c89ad946923aab407f'\
                      .format(dataframe.loc[park,'location.lat'],dataframe.loc[park,'location.lng'])

In [129]:
get_row(breeze_input)

breezometer_aqi                                                                     74
breezometer_color                                                              #72C833
breezometer_description                                               Fair air quality
country_aqi                                                                         52
country_aqi_prefix                                                                    
country_color                                                                  #FFFF00
country_description                                               Moderate air quality
country_name                                                             United States
data_valid                                                                        True
datetime                                                           2017-05-14T14:01:11
dominant_pollutant_canonical_name                                                  no2
dominant_pollutant_description             

In [249]:
def breez_call(parks):
    breeze_df = pd.DataFrame()
    for park in range(len(parks)):
        request = Request('https://api.breezometer.com/baqi/?lat={}&lon={}&key=932dc0cc968842c89ad946923aab407f'.format(parks.loc[park,'location.lat'],parks.loc[park,'location.lng']))
        response = urlopen(request)
        info = response.read()
        row = pd.read_json(info, typ='series', orient='values')
        breeze_df = pd.concat([breeze_df,row], axis=1, ignore_index=True )
        full_df = pd.concat([parks,breeze_df.T],axis=1)
    return full_df

In [215]:
full = breez_call(dataframe)

In [272]:
full

,beenHere.lastCheckinExpiredAt,categories,contact.facebook,contact.facebookName,contact.facebookUsername,contact.twitter,hasPerk,hereNow.count,hereNow.groups,hereNow.summary,...,country_color,country_description,country_name,data_valid,datetime,dominant_pollutant_canonical_name,dominant_pollutant_description,dominant_pollutant_text,key_valid,random_recommendations
0,0,"[{u'pluralName': u'Basketball Courts', u'prima...",104475634308,Bartow-Pell Mansion Museum,BartowPell,nycparks,False,0,[],Nobody here,...,#00E400,Good air quality,United States,True,2017-05-14T15:10:18,no2,Nitrogen dioxide,{u'main': u'The dominant pollutant is nitrogen...,True,{u'inside': u'The amount of pollutants in the ...
1,0,"[{u'pluralName': u'Playgrounds', u'primary': T...",NaN,NaN,NaN,NaN,False,0,[],Nobody here,...,#00E400,Good air quality,United States,True,2017-05-14T15:10:18,no2,Nitrogen dioxide,{u'main': u'The dominant pollutant is nitrogen...,True,{u'inside': u'The air quality is still good - ...
2,0,"[{u'pluralName': u'Parks', u'primary': True, u...",NaN,NaN,NaN,NaN,False,0,[],Nobody here,...,#00E400,Good air quality,United States,True,2017-05-14T15:10:18,o3,Ozone,{u'main': u'The dominant pollutant is ozone (O...,True,{u'inside': u'The air quality is still good - ...
3,0,"[{u'pluralName': u'Parks', u'primary': True, u...",NaN,NaN,NaN,nycparks,False,1,"[{u'count': 1, u'items': [], u'type': u'others...",One other person is here,...,#00E400,Good air quality,United States,True,2017-05-14T15:10:18,o3,Ozone,{u'main': u'The dominant pollutant is ozone (O...,True,{u'inside': u'The air quality is still good - ...
4,0,[{u'pluralName': u'Residential Buildings (Apar...,NaN,NaN,NaN,NaN,False,0,[],Nobody here,...,#00E400,Good air quality,United States,True,2017-05-14T15:10:18,no2,Nitrogen dioxide,{u'main': u'The dominant pollutant is nitrogen...,True,{u'inside': u'The amount of pollutants in the ...


## Downloading NYC neighborhoods and filtering out all but manhattan

In [ ]:
# Original data source: https://geo.nyu.edu/catalog/nyu_2451_34572

# Download the data 
!wget -O nyc_neighborhoods.csv --quiet https://ibm.box.com/shared/static/vgmf4nauors62vyzv7mg2ul4fpzvomen.csv

import pandas as pd

nyc = pd.read_csv("nyc_neighborhoods.csv")

nyc[['location.lat','location.lng']] = nyc[['latitude','longitude']]
nyc.drop(['latitude','longitude'],axis=1,inplace=True)
nyc = nyc[nyc['borough']=='Manhattan']
nyc.reset_index(inplace=True, drop=True)
nyc.drop('index',axis=1,inplace=True)

### Run breezometer function to get air quality for all manhattan neighborhoods

In [316]:
#manhattan_air = breez_call(nyc)

# EcoSquare Demo

In [280]:
def foursquare_call(lat,lon,k):
    fs_api_call = "https://api.foursquare.com/v2/venues/search?client_id=IRTQBGCEC2CBZPY3JUR35GNMFP2TBEE5A3MJSRFRE0GZCHXX&"+\
    "client_secret=ICGW5IXET5QQLCBUJIQX4T4Q30SJB2YHTOVF1OF13JVNCPD4&v=20170511&ll={},{}&query=park&limit={}&radius=3000".format(lat,lon,k)
    
    results = requests.get(fs_api_call).json()
    # assign relevant part of JSON to venues
    venues = results["response"]["venues"]

    # tranform venues into a dataframe
    dataframe = json_normalize(venues)
    return dataframe

In [281]:
def breez_call(parks):
    breeze_df = pd.DataFrame()
    for park in range(len(parks)):
        request = Request('https://api.breezometer.com/baqi/?lat={}&lon={}&key=932dc0cc968842c89ad946923aab407f'.format(parks.loc[park,'location.lat'],parks.loc[park,'location.lng']))
        response = urlopen(request)
        info = response.read()
        row = pd.read_json(info, typ='series', orient='values')
        breeze_df = pd.concat([breeze_df,row], axis=1, ignore_index=True )
        full_df = pd.concat([parks,breeze_df.T],axis=1)
    return full_df

# Fresno California 
### latitude:
36.785471
### longitude:
-119.820120

# Bedford-Stuyvesant Brooklyn
### latitude:
40.696631
### longitude:
-73.948966

# Chicago
### latitude:
41.8781
### longitude:
-87.6298

In [310]:
print 'Where are you?'
latitude = float(raw_input("Latitude: "))
longitude = float(raw_input("Longitude: "))


Where are you?
Latitude: 41.8781
Longitude: -87.6298


In [311]:
full = breez_call(foursquare_call(latitude,longitude,5))


In [312]:
park_map = folium.Map(location=[latitude,longitude], zoom_start=12, tiles = 'Stamen Toner')

for HOOD, LAT, LON, AQI, DESC, COLOR in zip(full.name, full['location.lat'], full['location.lng'], full['breezometer_aqi'],
                              full['breezometer_description'], full['breezometer_color']):
    folium.CircleMarker([LAT, LON], 
                        popup = str(HOOD)+': '+ str(DESC) +' '+ str(AQI), #text
                        radius = 15,
                       color = COLOR, fill_color = COLOR).add_to(park_map)


park_map


In [314]:
map_nyc = folium.Map(location=[40.765937,-73.977304], zoom_start=12, tiles = 'Stamen Toner')

for HOOD, LAT, LON, AQI, DESC, COLOR in zip(manhattan_air.neighborhood, manhattan_air['location.lat'], manhattan_air['location.lng'], manhattan_air['breezometer_aqi'],
                              manhattan_air['breezometer_description'], manhattan_air['breezometer_color']):
    folium.CircleMarker([LAT, LON], 
                        popup = str(HOOD)+': '+ str(DESC) +' '+ str(AQI), #text
                        radius = 15,
                       color = COLOR, fill_color=COLOR).add_to(map_nyc)


map_nyc


